# NLP基礎 (2): 単語埋め込み (Word Embeddings) - word2vec & GloVe

このノートブックでは、自然言語処理 (NLP) において単語の意味を捉えるための強力な手法である**単語埋め込み (Word Embeddings)** について学びます。
特に、代表的な手法である **word2vec (Skip-gram, CBOW)** と **GloVe (Global Vectors for Word Representation)** の基本的なアイデア、アーキテクチャ、そしてそれらがどのようにして単語の分散表現を獲得するかを解説します。
NumPyを使って主要な計算の概念を実装し、その後Gensimライブラリなどを用いて事前学習済みモデルを実際に扱ってみます。

**参考論文:**
*   Mikolov, T., Chen, K., Corrado, G., & Dean, J. (2013). Efficient estimation of word representations in vector space. *arXiv preprint arXiv:1301.3781*. (word2vecの初期の論文)
*   Mikolov, T., Sutskever, I., Chen, K., Corrado, G. S., & Dean, J. (2013). Distributed representations of words and phrases and their compositionality. In *Advances in neural information processing systems* (Vol. 26). (Skip-gram拡張版)
*   Pennington, J., Socher, R., & Manning, C. D. (2014). Glove: Global vectors for word representation. In *Proceedings of the 2014 conference on empirical methods in natural language processing (EMNLP)* (pp. 1532-1543). (GloVe論文)

**このノートブックで学ぶこと:**
1.  単語の分散表現と分布仮説の概念。
2.  word2vec (Skip-gram, CBOW) の仕組みと学習方法（特にNegative Sampling）。
3.  GloVeの仕組みと、共起行列の統計情報をどのように利用するか。
4.  NumPyによる主要な計算の概念実装。
5.  単語ベクトルの評価方法（類似度、アナロジー）。
6.  Gensimライブラリを使った事前学習済み単語ベクトルの利用。

**前提知識:**
*   テキスト前処理、Bag-of-Words、TF-IDFの基本的な理解（前のノートブック）。
*   ニューラルネットワークの基本的な概念（重み、活性化関数、損失関数など）。
*   NumPyの操作と線形代数の基礎。

## 1. 必要なライブラリのインポート

In [27]:
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import math

# Gensimは事前学習済みモデルのロードや学習に便利です
!pip install gensim
import gensim
from gensim.models import KeyedVectors

## 2. 単語埋め込みの基本概念

前のノートブックで学んだBoWやTF-IDFは、単語の出現頻度に基づいてテキストを表現しましたが、以下の限界がありました。
*   **語順の無視:** 単語の並び順が持つ意味を捉えられない。
*   **意味の類似性の欠如:** 「猫」と「犬」は意味的に近いですが、BoWやTF-IDFでは異なるインデックスを持つ独立した特徴として扱われ、その類似性は直接表現されません。
*   **高次元性とスパース性:** 語彙数が大きいとベクトルが非常に高次元かつ疎になりがち。

**単語埋め込み (Word Embeddings)** は、これらの問題を解決するために、各単語を比較的低次元（例: 50〜300次元程度）の**密なベクトル (dense vector)** で表現する手法です。このベクトルは**分散表現 (distributed representation)** とも呼ばれ、単語の意味がベクトルの複数の要素に分散して表現されることを意味します。

### 2.1 分布仮説 (Distributional Hypothesis)

多くの単語埋め込み手法の根底にあるのは、「**ある単語の意味は、その単語の周辺に出現する単語によって特徴づけられる**」という分布仮説です。
つまり、「猫」という単語がよく「かわいい」「鳴く」「ペット」といった単語と共に出現し、「犬」も同様の文脈で使われるならば、「猫」と「犬」は意味的に近いと判断できます。
単語埋め込みは、この文脈情報を利用して、意味的に類似した単語がベクトル空間上で近くに配置されるように学習します。

### 2.2 ベクトル空間モデル

学習された単語ベクトルは、多次元のベクトル空間を形成します。この空間では、以下のような興味深い性質が見られることがあります。
*   **類似性:** 意味的に似た単語同士のベクトルは、コサイン類似度などで測ると近くなります。
*   **アナロジー:** 「王様 - 男性 + 女性 = 女王様」のような、単語間の関係性がベクトル演算として表現できる場合があります ($vec(\text{king}) - vec(\text{man}) + vec(\text{woman}) \approx vec(\text{queen})$)。

## 3. Word2vec (Mikolov et al., 2013)

Word2vecは、GoogleのTomas Mikolovらによって提案された、効率的に高品質な単語ベクトルを学習する手法群です。主に以下の2つのモデルアーキテクチャがあります。

### 3.1 Skip-gram モデル

*   **アイデア:** ある中心語が与えられたときに、その周辺に出現する単語（コンテキストワード）を予測するように学習します。
*   **アーキテクチャ (論文 Figure 1):**
    1.  入力: 中心語 $w_I$ (通常はOne-Hotベクトルで表現)。
    2.  埋め込み層 (入力重み行列 $W$): $w_I$ を低次元の単語ベクトル $v_{w_I}$ に変換。
    3.  出力層 (出力重み行列 $W'$): $v_{w_I}$ から、各コンテキストウィンドウ内の各位置 $c$ における周辺語 $w_{I+c}$ の出現確率 $P(w_{I+c} | w_I)$ を予測します。
*   **学習目的 (論文式(1)の元):**
    与えられた中心語 $w_t$ に対するコンテキストウィンドウ内の各周辺語 $w_{t+j}$ ($-C \le j \le C, j \neq 0$) の対数尤度の平均を最大化します。
    $\frac{1}{T} \sum_{t=1}^{T} \sum_{-C \le j \le C, j \neq 0} \log P(w_{t+j} | w_t)$
    ここで、$P(w_O | w_I) = \frac{\exp(v_{w_O}'^T v_{w_I})}{\sum_{w=1}^{V} \exp(v_w'^T v_{w_I})}$ （Softmax関数）。$v_w$ は入力ベクトル、$v_w'$ は出力ベクトル。
    しかし、このSoftmaxの計算は語彙サイズ $V$ が大きいと非常にコストが高くなります。

### 3.2 CBOW (Continuous Bag-of-Words) モデル

*   **アイデア:** ある単語の周辺のコンテキストワードが与えられたときに、その中心語を予測するように学習します。
*   **アーキテクチャ:**
    1.  入力: 中心語の周辺にある複数のコンテキストワード（One-Hotベクトル）。
    2.  埋め込み層: 各コンテキストワードを単語ベクトルに変換し、それらを平均化（または合計）してコンテキストベクトルを作成。
    3.  出力層: コンテキストベクトルから中心語の出現確率を予測。
*   **学習目的:** Skip-gramと同様に、Softmaxの計算コストが問題となります。

### 3.3 学習効率化の手法

Word2vecの論文では、Softmax計算のボトルネックを解消するために、以下の効率的な学習手法が提案されました。

*   **Hierarchical Softmax:** 語彙内の単語を二分木（例: ハフマン木）で表現し、各単語へのパスを確率的にたどることで、計算量を $O(V)$ から $O(\log V)$ に削減します。
*   **Negative Sampling (NEG):**
    *   **アイデア:** 完全なSoftmaxを計算する代わりに、真の周辺語（ポジティブサンプル）と、ランダムに選んだいくつかの「ノイズ」単語（ネガティブサンプル）を識別する二値分類問題として学習します。
    *   **目的関数 (論文式(4)の元):**
        $\log \sigma(v_{w_O}'^T v_{w_I}) + \sum_{k=1}^{K} \mathbb{E}_{w_k \sim P_n(w)} [\log \sigma(-v_{w_k}'^T v_{w_I})]$
        ここで、$w_O$ は真の周辺語、$v_{w_I}$ は中心語の入力ベクトル、$v_w'$ は単語の出力ベクトル、$\sigma$ はSigmoid関数、$P_n(w)$ はノイズ分布（通常は単語の出現頻度の3/4乗に比例）、$K$ はネガティブサンプルの数です。
    *   この方法は、Hierarchical Softmaxよりも実装が簡単で、特に低頻度語のベクトル品質が良い傾向があります。

*   **Subsampling of Frequent Words (論文式(5)):**
    *   "the", "a" のような非常に頻繁に出現する単語は、情報量が少なく、学習を遅くする原因になります。
    *   これらの頻出語を一定の確率で訓練データから間引く（サブサンプリングする）ことで、学習速度を向上させ、より希少な単語の表現学習を促進します。
    *   単語 $w_i$ を破棄する確率 $P(w_i) = 1 - \sqrt{\frac{t}{f(w_i)}}$。ここで $f(w_i)$ は単語 $w_i$ の頻度、$t$ は閾値（例: $10^{-5}$）。

### 3.4 NumPyによるSkip-gram with Negative Sampling の主要計算（概念実装）

ここでは、中心語ベクトルと、ある（ポジティブまたはネガティブな）周辺語ベクトルが与えられたときに、そのペアのスコア（Sigmoid関数の入力）を計算する部分と、Negative Samplingの目的関数の一部を概念的に実装します。
完全な学習ループは複雑なので省略します。

In [28]:
def initialize_word_vectors(vocab_size, embedding_dim, random_seed=42):
    '''単語ベクトルを初期化
    Parameters:
        vocab_size (int): 語彙のサイズ
        embedding_dim (int): 埋め込み次元数
    Returns:
        W_in (np.ndarray): 中心語ベクトル
        W_out (np.ndarray): 周辺語ベクトル
    '''
    if random_seed is not None:
        np.random.seed(random_seed)

    # 中心語ベクトル
    # 一様乱数で初期化されることが多い
    limit = np.sqrt(1.0 / embedding_dim)
    W_in = np.random.uniform(-limit, limit, (vocab_size, embedding_dim))

    # 周辺語ベクトル
    # 本来はゼロで初期化されることが多い
    W_out = np.random.uniform(-limit, limit, (vocab_size, embedding_dim))

    return W_in, W_out

def skipgram_score(center_word, context_word):
    '''中心語と周辺語のスコアを計算
    Parameters:
        center_word (np.ndarray): 中心語のベクトル
        context_word (np.ndarray): 周辺語のベクトル
    Returns:
        score (float): 中心語と周辺語のスコア
    '''
    # v_context^T * v_center
    return np.dot(center_word, context_word)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [29]:
# テスト用のパラメータ
vocab_s = 10
embed_d = 5
W_input, W_output = initialize_word_vectors(vocab_s, embed_d)

center_word_idx = 0 # 語彙の0番目の単語を中心語とする
v_center = W_input[center_word_idx]

# ポジティブサンプル (例: 語彙の1番目の単語が実際の周辺語)
positive_context_idx = 1
v_context_positive = W_output[positive_context_idx]
score_positive = skipgram_score(v_center, v_context_positive)
prob_positive = sigmoid(score_positive)
print(f"Positive sample: Center='{center_word_idx}', Context='{positive_context_idx}'")
print(f"  Score: {score_positive:.4f}, Probability (sigmoid): {prob_positive:.4f}")
# 目的関数項 (ポジティブ): log(prob_positive)

# ネガティブサンプル (例: 語彙の5番目の単語をノイズとしてサンプリング)
K_negative_samples = 3
negative_context_indices = np.random.choice(vocab_s, size=K_negative_samples, replace=False) # ポジティブと重複しないように
print(f"\nNegative samples (indices): {negative_context_indices}")

loss_term_negative_sum = 0
for neg_idx in negative_context_indices:
    if neg_idx == positive_context_idx: continue # もし重複したらスキップ
    v_context_negative = W_output[neg_idx]
    score_negative = skipgram_score(v_center, v_context_negative)
    prob_negative_undesired = sigmoid(score_negative) # この確率が高いと困る
    # 目的関数項 (ネガティブ): log(1 - prob_negative_undesired) = log(sigmoid(-score_negative))
    loss_term_negative = np.log(sigmoid(-score_negative) + 1e-9) # log(0)回避
    loss_term_negative_sum += loss_term_negative
    print(f"  Negative: Context='{neg_idx}', Score: {score_negative:.4f}, Prob(undesired): {prob_negative_undesired:.4f}, log(1-Prob): {loss_term_negative:.4f}")

# 実際の目的関数はこれらの合計を最大化
# J = log σ(v_wO^T v_wI) + Σ E [log σ(-v_wk^T v_wI)]

Positive sample: Center='0', Context='1'
  Score: -0.2350, Probability (sigmoid): 0.4415

Negative samples (indices): [1 8 5]
  Negative: Context='8', Score: 0.0937, Prob(undesired): 0.5234, log(1-Prob): -0.7411
  Negative: Context='5', Score: 0.0903, Prob(undesired): 0.5226, log(1-Prob): -0.7393


## 4. GloVe (Global Vectors for Word Representation) (Pennington et al., 2014)

GloVeは、スタンフォード大学のPenningtonらによって提案された単語埋め込み手法です。
Word2vecが局所的なコンテキストウィンドウ内の情報に基づいて学習するのに対し、GloVeは**コーパス全体の単語-単語共起行列 (word-word co-occurrence matrix)** の統計情報を直接利用して単語ベクトルを学習します。

### 4.1 基本的なアイデア

GloVeの核心的なアイデアは、単語間の意味的な関係性が、共起確率の**比率**に現れるという考察に基づいています。
例えば、論文Table 1では、「ice」と「steam」という単語を考えます。
*   $P(k | \text{ice})$: 「ice」の文脈で単語 $k$ が出現する確率
*   $P(k | \text{steam})$: 「steam」の文脈で単語 $k$ が出現する確率

もし単語 $k$ が「solid」のように「ice」に強く関連し「steam」にあまり関連しない場合、$P(\text{solid} | \text{ice}) / P(\text{solid} | \text{steam})$ の比率は大きくなります。
逆に、$k$ が「gas」のように「steam」に強く関連する場合は比率が小さくなります。
$k$ が「water」のように両方に関連する場合や、「fashion」のようにどちらにもあまり関連しない場合は、比率は1に近くなります。

GloVeは、この共起確率の比率を、単語ベクトル間の演算でモデル化しようとします。
最も一般的な形式として、単語 $i$ と単語 $j$ のベクトル $w_i, w_j$、およびコンテキスト単語 $k$ のベクトル $\tilde{w}_k$ を用いて、以下の関係式を満たすように学習します (論文式(7)の元)。

$w_i^T \tilde{w}_k + b_i + \tilde{b}_k = \log(X_{ik})$

ここで、
*   $w_i, w_j$: 中心語の単語ベクトル
*   $\tilde{w}_k$: コンテキスト単語のベクトル（中心語ベクトルとは別に学習）
*   $b_i, \tilde{b}_k$: それぞれ中心語とコンテキスト単語のバイアス項
*   $X_{ik}$: 単語 $i$ の文脈で単語 $k$ が共起した回数（共起行列の要素）

### 4.2 損失関数

上記のモデルを満たすように、以下の重み付き最小二乗誤差を損失関数として定義します (論文式(8))。

$J = \sum_{i,j=1}^{V} f(X_{ij}) (w_i^T \tilde{w}_j + b_i + \tilde{b}_j - \log(X_{ij}))^2$

ここで、
*   $V$: 語彙サイズ
*   $f(X_{ij})$: 重み付け関数。共起回数 $X_{ij}$ が非常に大きい場合にその影響を抑え、また共起回数が0の場合に $\log(0)$ となるのを避けるための関数。論文では以下のような関数が提案されています (論文式(9))。
    $f(x) = \begin{cases} (x/x_{\max})^\alpha & \text{if } x < x_{\max} \\ 1 & \text{otherwise} \end{cases}$
    ($x_{\max}$ は通常100程度、$\alpha$ は通常0.75)

GloVeは、この損失関数 $J$ を確率的勾配降下法（AdaGradなど）で最小化することで、単語ベクトル $w$ とコンテキストベクトル $\tilde{w}$、およびバイアス $b, \tilde{b}$ を学習します。
最終的な単語ベクトルとしては、$w$ と $\tilde{w}$ の和 $W = w + \tilde{w}$ を使うことが推奨されています。

### 4.3 NumPyによるGloVe損失関数の主要部分（概念実装）

In [30]:
def glove_weighting(x, x_max=100, alpha=0.75):
    '''GloVeの重み付け関数'''
    if x > x_max:
        return 1.0
    else:
        return (x / x_max) ** alpha
    
def glove_loss_term(w_i, w_tilde_j, b_i, b_tilde_j, log_x_ij, x_ij_count, x_max=100, alpha=0.75):
    '''GloVeの損失項を計算'''
    weight = glove_weighting(x_ij_count, x_max, alpha)
    inner_term = np.dot(w_i, w_tilde_j) + b_i + b_tilde_j - log_x_ij
    return weight * (inner_term ** 2)

In [31]:
# テスト用パラメータ
vocab_s_glove = 10
embed_d_glove = 5
W_glove, W_tilde_glove = initialize_word_vectors(vocab_s_glove, embed_d_glove, random_seed=50) # 中心語とコンテキスト語用
b_glove = np.random.randn(vocab_s_glove) * 0.01 # 中心語バイアス
b_tilde_glove = np.random.randn(vocab_s_glove) * 0.01 # コンテキスト語バイアス

# ある単語ペア (i, j) を考える
i_idx = 0
j_idx = 1
X_ij_example_count = 50 # 単語iと単語jが50回共起したと仮定
log_X_ij_example = np.log(X_ij_example_count + 1e-9) # log(0)回避

loss_single_term = glove_loss_term(
    W_glove[i_idx], W_tilde_glove[j_idx], 
    b_glove[i_idx], b_tilde_glove[j_idx], 
    log_X_ij_example, X_ij_example_count
)
print(f"GloVe損失関数の一項 (X_ij={X_ij_example_count}): {loss_single_term:.4f}")

X_ij_rare_count = 1
log_X_ij_rare = np.log(X_ij_rare_count + 1e-9)
loss_rare_term = glove_loss_term(
    W_glove[i_idx], W_tilde_glove[j_idx], 
    b_glove[i_idx], b_tilde_glove[j_idx], 
    log_X_ij_rare, X_ij_rare_count
)
print(f"GloVe損失関数の一項 (X_ij={X_ij_rare_count}, 希少ペア): {loss_rare_term:.4f} (重みf(x)が小さくなる)")

GloVe損失関数の一項 (X_ij=50): 8.7652
GloVe損失関数の一項 (X_ij=1, 希少ペア): 0.0002 (重みf(x)が小さくなる)


## 5. 単語ベクトルの性質と評価

学習された単語ベクトルは、様々な興味深い性質を持ちます。

### 5.1 コサイン類似度による単語間の意味的・文法的類似性の測定
単語ベクトル間のコサイン類似度は、それらの単語の意味的な近さを測るためによく使われます。
$\text{similarity}(A, B) = \cos(\theta) = \frac{A \cdot B}{\|A\| \|B\|}$
値は-1から1の範囲を取り、1に近いほど類似度が高いことを意味します。

In [32]:
def cos_similarity(vec_a, vec_b):
    '''コサイン類似度を計算'''
    dot_product = np.dot(vec_a, vec_b)
    norm_a = np.linalg.norm(vec_a)
    norm_b = np.linalg.norm(vec_b)
    if norm_a == 0 or norm_b == 0:
        return 0.0
    return dot_product / (norm_a * norm_b)

In [33]:
# ダミーの単語ベクトルでテスト
vec_king = np.array([0.1, 0.5, -0.2])
vec_queen = np.array([0.15, 0.6, -0.15])
vec_man = np.array([0.8, 0.2, 0.1])
vec_woman = np.array([0.75, 0.3, 0.12])
vec_apple = np.array([-0.5, 0.1, 0.9])

print(f"Sim(king, queen): {cos_similarity(vec_king, vec_queen):.4f}")
print(f"Sim(king, man): {cos_similarity(vec_king, vec_man):.4f}")
print(f"Sim(king, apple): {cos_similarity(vec_king, vec_apple):.4f}")

Sim(king, queen): 0.9898
Sim(king, man): 0.3517
Sim(king, apple): -0.3177


### 5.2 アナロジータスク (Word Analogy)
単語ベクトルは、"king is to queen as man is to woman" のようなアナロジー関係をベクトル演算で捉えることができます。
具体的には、$vec(\text{king}) - vec(\text{man}) + vec(\text{woman})$ を計算し、この結果ベクトルに最もコサイン類似度が高い単語ベクトルを探すと、$vec(\text{queen})$ が得られることが期待されます。
これは、単語ベクトル空間が意味的な構造を線形的に保持していることを示唆します。

In [34]:
# アナロジーテスト: king - man + woman ~= queen
# (上記のダミーベクトルではあまり良い結果にならない可能性があります)
result_vector = vec_king - vec_man + vec_woman
print("vec(king) - vec(man) + vec(woman):\n", result_vector)

# このresult_vectorに最も近い単語ベクトルを語彙の中から探す
# (ここでは単純に queen との類似度を見る)
print(f"\nSim(result_vector, queen): {cos_similarity(result_vector, vec_queen):.4f}")
print(f"Sim(result_vector, king): {cos_similarity(result_vector, vec_king):.4f}")
print(f"Sim(result_vector, man): {cos_similarity(result_vector, vec_man):.4f}")
print(f"Sim(result_vector, woman): {cos_similarity(result_vector, vec_woman):.4f}")
print(f"Sim(result_vector, apple): {cos_similarity(result_vector, vec_apple):.4f}")
# 理想的には、queen との類似度が最も高くなる

vec(king) - vec(man) + vec(woman):
 [ 0.05  0.6  -0.18]

Sim(result_vector, queen): 0.9865
Sim(result_vector, king): 0.9907
Sim(result_vector, man): 0.2720
Sim(result_vector, woman): 0.3817
Sim(result_vector, apple): -0.1954


## 6. Gensimライブラリを使った事前学習済み単語ベクトルの利用

実際に高品質な単語ベクトルを学習するには、大規模なテキストコーパスと多くの計算時間が必要です。
幸いなことに、多くの研究機関や企業が事前学習済みの単語ベクトルモデルを公開しています。
Pythonライブラリの `gensim` を使うと、これらのモデルを簡単にロードして利用できます。

ここでは、Google Newsコーパス（約1000億単語）で学習されたword2vecモデル（約300万語彙、300次元）を例として使ってみます。
**注意:** このモデルファイル (`GoogleNews-vectors-negative300.bin.gz`) は非常に大きく（約3.4GB）、ダウンロードとロードに時間がかかります。初回実行時のみダウンロードが必要です。
Gensimがインストールされていない場合は、`pip install gensim` でインストールしてください。

In [35]:
model_name = 'word2vec-google-news-300'
word_vectors = None

print(f"\nLoading pre-trained word vectors: {model_name}")
import gensim.downloader as api
word_vectors = api.load(model_name)


Loading pre-trained word vectors: word2vec-google-news-300


In [36]:
print("\n--- Gensimを使った単語ベクトル操作 ---")

vector_king = word_vectors['king']
print('\nVector for "king" (first 5 dims):', vector_king[:5])
print('Vector dimensions:', vector_king.shape) # 300次元のはず

# 類似単語の検索
similar_to_king = word_vectors.most_similar('king', topn=5)
print("\nWords similar to 'king':", similar_to_king)
        
similar_to_woman = word_vectors.most_similar('woman', topn=5)
print("\nWords similar to 'woman':", similar_to_woman)

similar_to_france = word_vectors.most_similar('France', topn=5)
print("\nWords similar to 'France':", similar_to_france)

# アナロジータスク: king - man + woman = ?
analogy_result = word_vectors.most_similar(positive=['king', 'woman'], negative=['man'], topn=1)
print("\nAnalogy: king - man + woman = ", analogy_result) # queen が期待される

analogy_result_2 = word_vectors.most_similar(positive=['Paris', 'Germany'], negative=['France'], topn=1)
print("Analogy: Paris - France + Germany = ", analogy_result_2) # Berlin が期待される

# 単語間の類似度
sim_cat_dog = word_vectors.similarity('cat', 'dog')
print(f"\nSimilarity between 'cat' and 'dog': {sim_cat_dog:.4f}")
sim_cat_car = word_vectors.similarity('cat', 'car')
print(f"Similarity between 'cat' and 'car': {sim_cat_car:.4f}")


--- Gensimを使った単語ベクトル操作 ---

Vector for "king" (first 5 dims): [ 0.12597656  0.02978516  0.00860596  0.13964844 -0.02563477]
Vector dimensions: (300,)

Words similar to 'king': [('kings', 0.7138046622276306), ('queen', 0.6510956287384033), ('monarch', 0.6413194537162781), ('crown_prince', 0.6204220056533813), ('prince', 0.6159993410110474)]

Words similar to 'woman': [('man', 0.7664012312889099), ('girl', 0.7494640946388245), ('teenage_girl', 0.7336829900741577), ('teenager', 0.6317086219787598), ('lady', 0.6288785934448242)]

Words similar to 'France': [('French', 0.7000749707221985), ('extradites_Noriega', 0.6946742534637451), ('Belgium', 0.6933180689811707), ('Villebon_Sur_Yvette', 0.6776413321495056), ('PARIS_AFX_Gaz_de', 0.662800133228302)]

Analogy: king - man + woman =  [('queen', 0.7118193507194519)]
Analogy: Paris - France + Germany =  [('Berlin', 0.7644001841545105)]

Similarity between 'cat' and 'dog': 0.7609
Similarity between 'cat' and 'car': 0.2153


In [38]:
def analogy(a, b, c):
    try:
        result = word_vectors.most_similar(positive=[c, a], negative=[b], topn=1)
        return f"{a} - {b} + {c} → {result[0][0]} ({result[0][1]:.3f})"
    except KeyError as e:
        return f"単語が語彙にありません: {e}"


In [45]:
print(analogy("Windows", "Microsoft", "Apple"))   # → macOS
print(analogy("iPhone", "Apple", "Google"))       # → Pixel
print(analogy("Android", "Google", "Apple"))      # → iOS

Windows - Microsoft + Apple → Macs (0.674)
iPhone - Apple + Google → Gmail (0.655)
Android - Google + Apple → iPhone (0.676)


## 7. 考察

*   **単語埋め込みの威力:**
    *   Word2vecやGloVeのような手法は、単語を低次元の密なベクトル空間に写し、その空間内で意味的・文法的な関係性を捉えることを可能にしました。
    *   「王様 - 男性 + 女性 = 女王様」のようなアナロジーがベクトル演算で表現できることは、これらのモデルが単語の意味の非常に豊かな側面を学習していることを示唆しています。
*   **Word2vec vs GloVe:**
    *   **Word2vec (Skip-gram, CBOW):** 局所的なコンテキストウィンドウをスライドさせながら、中心語と周辺語の関係性を学習します。Negative Samplingなどの効率化手法が鍵となります。
    *   **GloVe:** コーパス全体の単語-単語共起行列の統計情報を直接モデル化しようとします。共起確率の比率に着目し、それをベクトル間の内積で再現するように学習します。
    *   どちらの手法も高品質な単語ベクトルを生成できますが、学習データやハイパーパラメータによって性能が変動することがあります。GloVeは全体の統計情報を使うため、より大域的な関係性を捉えやすいと言われることもあります。
*   **限界と発展:**
    *   **未知語 (OOV):** word2vecやGloVeは、訓練時に語彙に含まれなかった単語に対してはベクトルを生成できません。
    *   **文脈依存性の欠如:** これらの手法で得られる単語ベクトルは、単語ごとに固定です。しかし、同じ単語でも文脈によって意味が異なる場合があります（例: "bank"）。
    *   **形態情報:** 単語の内部構造（接頭辞、接尾辞など）を考慮していません。
    これらの限界に対処するために、サブワード情報を利用するFastTextや、文脈に応じた単語表現を学習するELMo、BERT、GPTといったより高度なモデルが登場しました。

このノートブックでは、単語埋め込みの基本的な考え方と代表的な手法を学びました。これらの単語ベクトルは、NLPタスクにおける入力特徴量として非常に広く使われており、多くのモデルの性能向上に貢献しています。
次のステップでは、これらの単語埋め込みを実際にニューラルネットワークの入力として利用するモデル（例: テキスト分類用のCNNやRNN）や、サブワード分割について学ぶのが良いでしょう。